# RULER evaluation

In [4]:
import re 
import sys 
sys.path.append('..')
from utils.utils import read_jsonl
from utils.calibration_utils import auroc, ece, brier_score, nll

def ruler_eval_func(base_url, predict_url):
    base = read_jsonl(base_url)
    check = read_jsonl(predict_url)
    predicts = []
    targets = []
    conf = []
    for b, predict in zip(base, check):
        conf.append(int(predict['confidence'])*0.01)
        targets.append([p.lower() for p in predict['reference']])
        if b['task_label'] == 'cwe' or b['task_label'] == 'fwe' or b['task_label'] == 'vt':
            predicts.append([re.sub(r'[^a-zA-Z]', '', w).lower() for w in predict['completion'].split(" ")])
        elif b['task_label'] == 'niah_multikey_1' or b['task_label'] == 'niah_multikey_2' or b['task_label'] == 'niah_single_1' or b['task_label'] == 'niah_single_2':
            predicts.append([re.sub(r'[^0-9]', '', predict['completion']).lower()]) 
        elif b['task_label'] == 'niah_multikey_3' or b['task_label'] == 'niah_single_3':
            predicts.append([re.sub(r'[^a-zA-Z0-9-]', '', predict['completion']).lower()])
        elif b['task_label'] == 'niah_multiquery' or b['task_label'] == 'niah_multivalue':
            predicts.append([re.sub(r'[^0-9]', '', w).lower() for w in predict['completion'].split(" ")])
        else:
            predicts.append([predict['completion'].lower()])
    tf = []
    #new_p = []
    for b, p, t in zip(base, predicts, targets):
        p = [x for x in p if x != '']
        #new_p.append(p)
        if 'qa' in b['task_label']:
            if set(p).issubset(set(t)):
                tf.append(1)
            else:
                tf.append(0)
        else:
            try:
                if set(p) == set(t):
                    tf.append(1)
                else:
                    tf.append(0) 
            except:
                print(p)
    return tf, conf 

In [16]:
def print_results(tf, conf):
    print("AUROC: ", auroc(tf, conf))
    print("ACC: ", sum(tf)/len(tf))
    print("ECE: ", ece(tf, conf, n_bins=10))
    print("BS: ", brier_score(tf, conf))
    #print("NLL: ", nll(tf, conf))

In [19]:
tf, conf = ruler_eval_func(
    "../data/processed/ruler_4k_test.jsonl",
    "../logs/zero_shot_test_evals/llama_3b_ruler_4k_test.jsonl"
)
print_results(tf, conf)

AUROC:  0.6056796690307328
ACC:  0.7230769230769231
ECE:  0.25830769230769257
BS:  0.2595538461538462
